In [ ]:
pip install tensorflow

In [1]:
import pandas as pd

C:\Users\HP830G5\anaconda3\Lib\site-packages\pandas\core\computation\expressions.py:22: UserWarning: Pandas requires version '2.10.2' or newer of 'numexpr' (version '2.8.4' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\HP830G5\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:56: UserWarning: Pandas requires version '1.4.2' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
cars_cleaned = pd.read_csv('../../data/scrapped/voitures_clean.csv')
cars_cleaned.head()

,Ville,Marque,Modèle,Année-Modèle,Kilométrage,Type de carburant,Puissance fiscale,Boite de vitesses,Nombre de portes,Origine,Première main,Prix,Ville_grouped,Age_vehicule
0,Temara,Fiat,Punto,2007,224999.5,Diesel,5.0,Manuelle,5.0,WW au Maroc,Non,60000.0,Temara,18
1,Casablanca,Volkswagen,Touareg,2005,2499.5,Diesel,10.0,Automatique,5.0,WW au Maroc,Oui,90000.0,Casablanca,20
2,Dakhla,Toyota,Prado,2007,224999.5,Diesel,12.0,Manuelle,5.0,WW au Maroc,Non,97000.0,Autres,18
3,Khouribga,Volkswagen,Tiguan,2014,184999.5,Diesel,8.0,Automatique,5.0,Dédouanée,Non,255000.0,Autres,11
4,Meknès,Peugeot,308,2009,224999.5,Diesel,6.0,Manuelle,5.0,WW au Maroc,Non,85000.0,Meknès,16


In [3]:
df1 = cars_cleaned.copy()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import r2_score
from tensorflow.keras import layers

df1 = df1.drop(columns=['Ville',"Année-Modèle"])
df1['logPrix'] = np.log1p(df1['Prix'])

# Variables et target
X = df1.drop(['Prix','logPrix'], axis=1)
y = df1['logPrix']

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ONE-HOT ENCODING APRES SPLIT (propre)
ohe_cols = ['Boite de vitesses','Type de carburant','Origine','Première main',"Ville_grouped"]

X_train = pd.get_dummies(X_train, columns=ohe_cols, drop_first=True)
X_test  = pd.get_dummies(X_test,  columns=ohe_cols, drop_first=True)

# Align train & test
X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)

# TARGET ENCODING (propre)
for col in ['Marque','Modèle']:
    means = X_train.join(y_train)[[col,'logPrix']].groupby(col)['logPrix'].mean()

    X_train[col + '_te'] = X_train[col].map(means)

    global_mean = y_train.mean()
    X_test[col + '_te'] = X_test[col].map(means).fillna(global_mean)

# Supprimer colonnes originales
X_train = X_train.drop(columns=['Marque','Modèle'])
X_test  = X_test.drop(columns=['Marque','Modèle'])


# ==== STANDARDISATION UNIQUEMENT DES VARIABLES NUMÉRIQUES ====
num_cols = ['Kilométrage', 'Age_vehicule', 'Nombre de portes', 'Puissance fiscale','Marque_te','Modèle_te']
scaler = StandardScaler()
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

X_train_scaled[num_cols] = scaler.fit_transform(X_train[num_cols])
X_test_scaled[num_cols] = scaler.transform(X_test[num_cols])

model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)  
])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae']
)
history = model.fit(
    X_train_scaled, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=32,
    verbose=1
)
test_loss, test_mae = model.evaluate(X_test_scaled, y_test)
print("Erreur MAE (log-prix) :", test_mae)

y_pred_log = model.predict(X_test_scaled)
y_pred = np.expm1(y_pred_log)   
y_real = np.expm1(y_test)
mae_real = np.mean(np.abs(y_real - y_pred))
r2 = r2_score(y_real, y_pred)
print("Erreur moyenne en DH :", mae_real)
print("R² sur les prix réels :", r2)